In [1]:
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
from scipy.ndimage import zoom
import pandas as pd
from pipeline_setup import *
import time
from scipy.optimize import minimize

In [2]:
# Set file locations
advanced_settings_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTanjc08kc5vIN-icUMzMEGA9bJuDesLX8V_u2Ab6zSC4MOhLZ8Jrr18DL9o4ofKIrSq6FsJXhPWu3F/pub?gid=0&single=true&output=csv"

# Read in emissions from other sectors
sector_emissions_dict = set_sector_emissions_dict()
# Read in the advanced settings from the google sheet

params = read_advanced_settings(advanced_settings_url)

# Set the datablock
datablock_init = datablock_setup()

# Set the scenario parameters
params_baseline = set_baseline_scenario(params)

# Also add the baseline parameters to the datablock
datablock_init.update(params_baseline)

In [3]:
# Read parameter ranges from scenarios spreadsheet
ranges_worksheet_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vRXLuSuuxfTx1tUilnO1KojbaGiO-o-rtf1OtsQ0YHetV-OozWH1BXc7N-1Y9jG9Ue2ys7mcf-SzPc3/pub?gid=1034155472&single=true&output=csv"
ranges = pd.read_csv(ranges_worksheet_url, dtype='string', skiprows=2)
ranges.head()

,Name,pop_proj,yield_proj,elasticity,ruminant,dairy,pig_poultry,eggs,pulses,fruit_veg,...,livestock_yield,agroforestry,arable_soil_carbon,fossil_arable,nitrogen,vertical_farming,waste_BECCS,overseas_BECCS,DACCS,biochar
0,Min JPSarah1618 Thu19Jun25,Medium,0,0,-70,-60,-60,-60,0,-20,...,100,0,100,100,100,0,16,0,0,0
1,Max JPSarah1618 Thu19Jun25,Medium,0,0,0,15,15,15,500,500,...,100,100,100,100,100,60,16,0,0,0
2,Min LessMixedFarming,Medium,0,0,-70,-60,-60,-60,0,-20,...,100,0,100,100,100,0,16,0,0,0
3,Max LessMixedFarming,Medium,0,0,0,15,15,15,500,500,...,100,100,100,100,100,60,16,0,0,0


In [4]:
# Remove "Max " and "Min " prefixes and extract unique names
unique_names = ranges["Name"].dropna().str.replace(r"^(Max |Min )", "", regex=True).unique()
unique_names
# Create a dictionary to store the ranges
ranges_dict = {}

# Iterate over unique range names
for name in unique_names:
    # Filter rows corresponding to the current range name
    min_row = ranges[ranges["Name"] == f"Min {name}"].iloc[0]
    max_row = ranges[ranges["Name"] == f"Max {name}"].iloc[0]
    
    # Extract parameter ranges as tuples
    param_ranges = {
        col: (float(min_row[col]), float(max_row[col]))
        for col in ranges.columns[3:]  # Skip the "Name" column
        if pd.notna(min_row[col]) and pd.notna(max_row[col])  # Ensure values are not NaN
    }
    
    # Add to the dictionary
    ranges_dict[name] = param_ranges

list(ranges_dict.keys())

['JPSarah1618 Thu19Jun25', 'LessMixedFarming']

In [5]:
# Name the output parameters of the current calculator function 
# (ideally these would be set and returned by the calculator)
z1_name = "SSR weight"
z2_name = "SSR prot"
z3_name = "SSR fat"
z4_name = "SSR kcal"
z5_name = "emissions"
z6_name = "herd size"
z7_name = "animals"

z_names = [z1_name, z2_name, z3_name, z4_name, z5_name, z6_name, z7_name]

# Name the params we are going to vary
param_range_dict = {
    "ruminant":             (-100, 50),
    "pig_poultry":          (-100, 100),
    "fish_seafood":         (-100, 100),
    "dairy":                (-50, 100),
    "eggs":                 (-100, 100),
    "fruit_veg":            (-100, 500),
    "pulses":               (-100, 500),
    "meat_alternatives":    (0, 100),
    "dairy_alternatives":   (0, 100),
    "waste":                (0, 100),

    "foresting_pasture":    (10, 50),
    "land_BECCS":           (0, 100),
    "land_BECCS_pasture":   (0, 100),
    "horticulture":         (-100, 500),
    "pulse_production":     (-100, 500),
    "lowland_peatland":     (0, 100),
    "upland_peatland":      (0, 100),
    "pasture_soil_carbon":  (0, 100),
    "arable_soil_carbon":   (0, 100),
    "mixed_farming":        (0, 25),

    "silvopasture":         (0,100),
    "methane_inhibitor":    (0,100),
    "stock_density":        (-100,50),
    "manure_management":    (0,100),
    "animal_breeding":      (0,100),
    "fossil_livestock":     (0,100),

    "agroforestry":         (0,100),
    "nitrogen":             (0,100),
    "vertical_farming":     (0,100),
    "fossil_arable":        (0,100),

    "waste_BECCS":          (0,40)
           }

# List of parameter names and ranges
def names_bounds(param_range_dict):

    param_range_dict_with_range = {k: v for k, v in param_range_dict.items() if v[0] != v[1]}

    names_x = list(param_range_dict_with_range.keys())
    x_bounds = list(param_range_dict_with_range.values())

    names_fixed = [k for k, v in param_range_dict.items() if v[0] == v[1]]
    values_fixed = [v[0] for k, v in param_range_dict.items() if v[0] == v[1]]

    return names_x, x_bounds, names_fixed, values_fixed

# names_x, x_bounds = names_bounds(param_range_dict)
names_x, x_bounds, names_fixed, values_fixed = names_bounds(ranges_dict["JPSarah1618 Thu19Jun25"])
names_x

['ruminant',
 'dairy',
 'pig_poultry',
 'eggs',
 'pulses',
 'fruit_veg',
 'meat_alternatives',
 'dairy_alternatives',
 'waste',
 'foresting_pasture',
 'land_BECCS',
 'land_BECCS_pasture',
 'lowland_peatland',
 'upland_peatland',
 'horticulture',
 'pulse_production',
 'mixed_farming',
 'silvopasture',
 'stock_density',
 'agroforestry',
 'vertical_farming']

In [6]:
# Update baseline parameters
for k, v in zip(names_fixed, values_fixed):
    datablock_init[k] = v
    params_baseline[k] = v
    print(k, v)

elasticity 0.0
bdleaf_conif_ratio 75.0
pasture_soil_carbon 100.0
methane_inhibitor 100.0
manure_management 100.0
animal_breeding 100.0
fossil_livestock 100.0
livestock_yield 100.0
arable_soil_carbon 100.0
fossil_arable 100.0
nitrogen 100.0
waste_BECCS 16.0
overseas_BECCS 0.0
DACCS 0.0
biochar 0.0


In [7]:
# Create an class with anobjective function suitable for giving to the scipy minimizer
# Including a cache to avoid recomputing the same values
class FFCObjectiveWithCache:
    """A class to compute the objective function and constraints for FFC optimization with caching.
    Parameters:
        names_x (list): List of parameter names to be optimized.
        datablock_init (dict): Initial data block containing fixed parameters.
        params_default (dict): Default parameters for the optimization.
        sector_emissions_dict (dict): Dictionary containing emissions data for different sectors.
        verbosity (int): Level of verbosity for output messages.
    """
    def __init__(self, names_x, datablock_init, params_default, verbosity=0):
        self.names_x = names_x
        self.datablock_init = datablock_init
        self.params_default = params_default
        self._cache = {}
        self.verbosity = verbosity

        # Define the names of the z variables returned by the calculator
        self.z_names = ["SSR weight",
                   "SSR prot",
                   "SSR fat",
                   "SSR kcal",
                   "emissions",
                   "herd size",
                   "animals"]

    def _calculate(self, x_tuple, verbosity):
        
        x = list(x_tuple)
        # Only recompute if not already cached
        
        if x_tuple not in self._cache:

            # Update the parameters with the current values
            params = self.params_default.copy()
            for i_name, name_string in enumerate(self.names_x):
                params[name_string] = x[i_name]
            
            # Perform the SSR and emissions calculation
            z_val = run_calculator(self.datablock_init, params)

            # cached dict
            zval_dict = {zn: zv for zn, zv in zip(self.z_names, z_val)}

            # Store the results in the cache
            self._cache[x_tuple] = zval_dict

        # Print out what's going on 
        if (verbosity > 1):
            for i_name, name_string in enumerate(self.names_x):
                print(f"{name_string} = {x[i_name]:.10f}; ", end="")
            for i_name, name_string in enumerate(list(self._cache[x_tuple].keys())):
                print(f"{name_string} = {self._cache[x_tuple][name_string]:.10f}; ", end="")
            print()

        return self._cache[x_tuple]

    # Define the objective function for minimization
    def objective(self, x, z_name_requested, verbosity=None):
        x_tuple = tuple(x)
        if verbosity is None:
            verbosity = self.verbosity
        return self._calculate(x_tuple, verbosity)[z_name_requested]
    
    # Define the objective function for minimization
    def negative_objective(self, x, z_name_requested, verbosity=None):
        x_tuple = tuple(x)
        if verbosity is None:
            verbosity = self.verbosity
        return -self._calculate(x_tuple, verbosity)[z_name_requested]

    # Define the objective function for SSR constraint
    def positive_constraint(self, x, key, threshold, verbosity=None):
        x_tuple = tuple(x)
        if verbosity is None:
            verbosity = self.verbosity
        return self._calculate(x_tuple, verbosity)[key] - threshold
    
    def negative_constraint(self, x, key, threshold, verbosity=None):
        x_tuple = tuple(x)
        if verbosity is None:
            verbosity = self.verbosity
        return threshold - self._calculate(x_tuple, verbosity)[key]
    

In [8]:
# Do a test calculation
z_val_baseline = run_calculator(datablock_init, params_baseline)
#print(f"SSR = {SSR_result:.8f}; GHG = {emissions_result:.8f}")

for zn, zval in zip(z_names, z_val_baseline):
    print(f"{zn} = {zval:.8f}; ", end="")
# This is my checksum for debugging: 
# SSR = 0.67366432; GHG = 94.22954995

SSR weight = 0.67366432; SSR prot = 0.73314955; SSR fat = 0.63337477; SSR kcal = 0.68543863; emissions = 54.43513442; herd size = 9151744.10648140; animals = 222863917.05211931; 

In [ ]:
# Set tolerances, verbosity, and options for the minimizer
ffc_tol = 1e-6
options = {
    'disp': True,      # Show convergence messages
    'maxiter': 5000,     # Max number of iterations
    'rhobeg' : 10 # Reasonable step size (mostly they are percentages, so change by 10%)
}


In [ ]:
ffc_wrapper = FFCObjectiveWithCache(names_x, datablock_init, params_baseline, verbosity=2)

# z_name_requested = "SSR weight"
z_name_requested = "herd size"
z_name_requested = "animals"
x0 = [params_baseline[n] for n in names_x]
ffc_constraints = [{'type': 'ineq', 'fun': lambda x: ffc_wrapper.positive_constraint(x, "SSR weight", threshold=0.7201, verbosity=0)},
                   {'type': 'ineq', 'fun': lambda x: ffc_wrapper.positive_constraint(x, "SSR prot", threshold=0.7837, verbosity=0)},
                   {'type': 'ineq', 'fun': lambda x: ffc_wrapper.positive_constraint(x, "SSR fat", threshold=0.6770, verbosity=0)},
                   {'type': 'ineq', 'fun': lambda x: ffc_wrapper.positive_constraint(x, "SSR kcal", threshold=0.7327, verbosity=0)},
                   {'type': 'ineq', 'fun': lambda x: ffc_wrapper.negative_constraint(x, "emissions", threshold=0.0, verbosity=0)}]

result = minimize(
    lambda x: ffc_wrapper.negative_objective(x, z_name_requested),
    # lambda x: ffc_wrapper.objective(x, z_name_requested),
    x0,
    method='COBYLA',
    bounds=x_bounds,
    constraints=ffc_constraints,
    tol=ffc_tol,
    options=options
)

ruminant = 0.0000000000; dairy = 0.0000000000; pig_poultry = 0.0000000000; eggs = 0.0000000000; pulses = 0.0000000000; fruit_veg = 0.0000000000; meat_alternatives = 0.0000000000; dairy_alternatives = 0.0000000000; waste = 0.0000000000; foresting_pasture = 13.1700000000; land_BECCS = 0.0000000000; land_BECCS_pasture = 0.0000000000; lowland_peatland = 0.0000000000; upland_peatland = 0.0000000000; horticulture = 0.0000000000; pulse_production = 0.0000000000; mixed_farming = 0.0000000000; silvopasture = 0.0000000000; stock_density = 0.0000000000; agroforestry = 0.0000000000; vertical_farming = 0.0000000000; SSR weight = 0.6736643225; SSR prot = 0.7331495528; SSR fat = 0.6333747730; SSR kcal = 0.6854386315; emissions = 54.4351344249; herd size = 9151744.1064814031; animals = 222863917.0521193147; 
ruminant = 10.0000000000; dairy = 0.0000000000; pig_poultry = 0.0000000000; eggs = 0.0000000000; pulses = 0.0000000000; fruit_veg = 0.0000000000; meat_alternatives = 0.0000000000; dairy_alternativ

In [11]:
result

 message: Did not converge to a solution satisfying the constraints. See `maxcv` for magnitude of violation.
 success: False
  status: 4
     fun: -222863917.0521193
       x: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
    nfev: 10
   maxcv: 54.435134424867854

In [12]:
# The result is an OptimizeResult object
print("Optimization success:", result.success)
print("Message:", result.message)
print("Number of iterations:", result.nfev)
print("Optimal value of x:", result.x)
print("Minimum value of function:", result.fun)

Optimization success: False
Message: Did not converge to a solution satisfying the constraints. See `maxcv` for magnitude of violation.
Number of iterations: 10
Optimal value of x: [ 0.    0.    0.    0.    0.    0.    0.    0.    0.   13.17  0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
Minimum value of function: -222863917.0521193


In [13]:
print("Optimized parameters: ", result['x'])      # Optimal parameters
print("Optimized value: ", result['fun'])    # Minimum value of the objective
print("Was the minimization succesfull? ", result['success'])  # Boolean indicating if it was successful

Optimized parameters:  [ 0.    0.    0.    0.    0.    0.    0.    0.    0.   13.17  0.    0.
  0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
Optimized value:  -222863917.0521193
Was the minimization succesfull?  False


In [14]:
# Display the results
#z1_val, z2_val = list(ffc_results.values())[:2]
z1_val = ffc_wrapper.objective(result.x, "SSR weight")
z2_val = ffc_wrapper.objective(result.x, "emissions")
print(f"SSR weight = {z1_val:.8f}; emissions = {z2_val:.8f}")


ruminant = 0.0000000000; dairy = 0.0000000000; pig_poultry = 0.0000000000; eggs = 0.0000000000; pulses = 0.0000000000; fruit_veg = 0.0000000000; meat_alternatives = 0.0000000000; dairy_alternatives = 0.0000000000; waste = 0.0000000000; foresting_pasture = 13.1700000000; land_BECCS = 0.0000000000; land_BECCS_pasture = 0.0000000000; lowland_peatland = 0.0000000000; upland_peatland = 0.0000000000; horticulture = 0.0000000000; pulse_production = 0.0000000000; mixed_farming = 0.0000000000; silvopasture = 0.0000000000; stock_density = 0.0000000000; agroforestry = 0.0000000000; vertical_farming = 0.0000000000; SSR weight = 0.6736643225; SSR prot = 0.7331495528; SSR fat = 0.6333747730; SSR kcal = 0.6854386315; emissions = 54.4351344249; herd size = 9151744.1064814031; animals = 222863917.0521193147; 
ruminant = 0.0000000000; dairy = 0.0000000000; pig_poultry = 0.0000000000; eggs = 0.0000000000; pulses = 0.0000000000; fruit_veg = 0.0000000000; meat_alternatives = 0.0000000000; dairy_alternative

In [18]:
# Generate URL with output parameters
results_dict = {k: v for k, v in zip(result["x"], names_x)}

URL_name = "sarahjp-hack.streamlit.app/?"

for n, x in zip(names_x, result["x"]):

    print(f"{n} {x:.2f}")

    URL_name += f"{n}={x:.2f}&"

URL_name = URL_name[:-1]
# print(URL_name)

ruminant 0.00
dairy 0.00
pig_poultry 0.00
eggs 0.00
pulses 0.00
fruit_veg 0.00
meat_alternatives 0.00
dairy_alternatives 0.00
waste 0.00
foresting_pasture 13.17
land_BECCS 0.00
land_BECCS_pasture 0.00
lowland_peatland 0.00
upland_peatland 0.00
horticulture 0.00
pulse_production 0.00
mixed_farming 0.00
silvopasture 0.00
stock_density 0.00
agroforestry 0.00
vertical_farming 0.00
